In [23]:
import pandas as pd
import json
import requests
from tqdm import tqdm
from time import sleep

In [8]:
with open('pheno_events.json', 'r') as fh:
    data = json.load(fh)
df = pd.DataFrame(data)
df.head()

,score,month,phenophase_name,common_name,genus,species
0,0.633333,4,Breaking leaf buds,Allegheny serviceberry,amelanchier,laevis
1,0.300000,4,Flowers or flower buds,Allegheny serviceberry,amelanchier,laevis
2,0.750000,5,Increasing leaf size,Allegheny serviceberry,amelanchier,laevis
3,1.000000,5,Leaves,Allegheny serviceberry,amelanchier,laevis
4,0.083333,6,Fruits,Allegheny serviceberry,amelanchier,laevis


In [12]:
def make_the_call(endpoint, **kwargs):
    url = f'https://api.inaturalist.org/v1/{endpoint}'
    if kwargs:
        query = '&'.join([f'{key}={value}' for key, value in kwargs.items()])
        url = f'{url}?{query}'
    return requests.request(
        url=url,
        method='GET'
    ).json()

make_the_call('taxa', q='Amelanchier-laevis')

{'total_results': 2,
 'page': 1,
 'per_page': 30,
 'results': [{'observations_count': 1804,
   'taxon_schemes_count': 3,
   'ancestry': '48460/47126/211194/47125/47124/47132/47148/922110/723302/49230',
   'is_active': True,
   'flag_counts': {'unresolved': 0, 'resolved': 1},
   'wikipedia_url': 'http://en.wikipedia.org/wiki/Amelanchier_laevis',
   'current_synonymous_taxon_ids': None,
   'iconic_taxon_id': 47126,
   'rank_level': 10,
   'taxon_changes_count': 0,
   'atlas_id': None,
   'complete_species_count': None,
   'parent_id': 49230,
   'name': 'Amelanchier laevis',
   'rank': 'species',
   'extinct': False,
   'id': 54814,
   'default_photo': {'square_url': 'https://inaturalist-open-data.s3.amazonaws.com/photos/125691428/square.jpeg',
    'attribution': 'no hay derechos reservados, uploaded by Étienne Lacroix-Carignan',
    'flags': [],
    'medium_url': 'https://inaturalist-open-data.s3.amazonaws.com/photos/125691428/medium.jpeg',
    'id': 125691428,
    'license_code': 'cc0',

In [24]:
df['name'] = df['genus'] + '-' + df['species']
rows = []
missed = set()
for name in tqdm(df['name'].unique()):
    try:
        options = make_the_call('taxa', q=name)['results']
        rows.append({
            'genus': name.split('-')[0].lower(),
            'species': name.split('-')[1].lower(),
            'iNatID': options[0]['id']
        })
    except:
        missed.add(name)
    sleep(1)

100%|██████████| 105/105 [02:34<00:00,  1.47s/it]


In [28]:
with open('inat_info.json', 'w') as fh:
    json.dump(rows, fh)